## R 스타일 모형 정의

- Patsy 패키지
- R-style formula 연산자
- Dmatrix 변환
- 변수 보호
- 다항 선형 회귀 (polynomial regression)
- 카테고리 변수 인코딩
- OLS 모형 정의 문자열

---

### 00. Setting

In [1]:
import statsmodels.api as sm
import statsmodels.stats.api as sms
from patsy import dmatrix
import scipy as sp
import seaborn as sns
import statsmodels.api as sm
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

/usr/local/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


---

### 01. Patsy 패키지 

#### 1. 정의

회귀분석 전처리를 위한 패키지로 데이터프레임을 가공하여 인코딩, 변환 등을 쉽게 해주는 기능을 제공

#### 2. dmatrix 함수

1) 함수 설명

- pasty 패키지 있는 함수


- dmatrix라는 명령을 사용하면 실험설계행렬(experiment design matrix)을 간단히 만들수 있음

```
data_transformed = dmatrix(formula, data)
```

- formula : 모형 정의 문자열 
- data : 원데이터 
- data_transformed : formula에서 지정한대로 변환된 데이터를 출력

2) 예시

In [2]:
# 데이터 생성
np.random.seed(0)
x1 = np.random.rand(5) + 10
x2 = np.random.rand(5) * 10
y = x1 + 2 * x2 + np.random.randn(5) # y = x1 + 2*x2 + random_number

df1 = pd.DataFrame(x1, columns=["x1"])
df2 = pd.DataFrame(np.array([x1, x2]).T, columns=["x1", "x2"])
df = pd.DataFrame(np.array([x1, x2, y]).T, columns=["x1", "x2", "y"])

In [3]:
df1 # df1 데이터프레임 확인

,x1
0,10.548814
1,10.715189
2,10.602763
3,10.544883
4,10.423655


In [4]:
df2 # df2 데이터프레임 확인

,x1,x2
0,10.548814,6.458941
1,10.715189,4.375872
2,10.602763,8.917730
3,10.544883,9.636628
4,10.423655,3.834415


In [5]:
df # df 데이터 확인

,x1,x2,y
0,10.548814,6.458941,23.610739
1,10.715189,4.375872,20.921207
2,10.602763,8.917730,29.199261
3,10.544883,9.636628,29.939813
4,10.423655,3.834415,18.536348


자동 Augmentation 

In [6]:
dmatrix("x1", df1) # df1에 있는 x1 컬럼에 augmentaion을 적용

DesignMatrix with shape (5, 2)
  Intercept        x1
          1  10.54881
          1  10.71519
          1  10.60276
          1  10.54488
          1  10.42365
  Terms:
    'Intercept' (column 0)
    'x1' (column 1)

---

### 02. R-style formula 연산자

#### 1. 종류

| 기호 | 설명 |
|-|-|
|+0, -1| 상수항을 제외|
|`+`| 설명 변수 추가 |
|`-`| 설명 변수 제거 |
|`:`| interaction (곱) |
|`*`| `a*b = a + b + a:b` |
|`/`| `a/b = a + a:b` |
|`~`| 종속 - 독립 관계 |

1) - 1 또는 + 0 : 상수항을 제외하고자 하는 경우 (augmentation 제거)

In [7]:
dmatrix("x1 - 1") # 제거된 것을 확인

DesignMatrix with shape (5, 1)
        x1
  10.54881
  10.71519
  10.60276
  10.54488
  10.42365
  Terms:
    'x1' (column 0)

In [8]:
dmatrix("x1 + 0") # 제거된 것을 확인

DesignMatrix with shape (5, 1)
        x1
  10.54881
  10.71519
  10.60276
  10.54488
  10.42365
  Terms:
    'x1' (column 0)

2) + : 두 개 이상의 데이터를 포함하는 경우 (concat같은 느낌)

In [9]:
dmatrix("x1 + x2") # 데이터가 합쳐져 있음

DesignMatrix with shape (5, 3)
  Intercept        x1       x2
          1  10.54881  6.45894
          1  10.71519  4.37587
          1  10.60276  8.91773
          1  10.54488  9.63663
          1  10.42365  3.83442
  Terms:
    'Intercept' (column 0)
    'x1' (column 1)
    'x2' (column 2)

3) `:` : 데이터의 곱, 교호작용을 확인하기 위해 사용

In [10]:
dmatrix("x1 : x2 -1")

DesignMatrix with shape (5, 1)
      x1:x2
   68.13417
   46.88830
   94.55258
  101.61711
   39.96862
  Terms:
    'x1:x2' (column 0)

4) `*` : 데이터1 , 데이터2, 데이터1*데이터2

In [11]:
dmatrix("x1 * x2 - 1") # x1, x2, x1*x2 형태

DesignMatrix with shape (5, 3)
        x1       x2      x1:x2
  10.54881  6.45894   68.13417
  10.71519  4.37587   46.88830
  10.60276  8.91773   94.55258
  10.54488  9.63663  101.61711
  10.42365  3.83442   39.96862
  Terms:
    'x1' (column 0)
    'x2' (column 1)
    'x1:x2' (column 2)

5) `/` : a / b = a + a : b

In [12]:
dmatrix("x1 / x2 - 1") # x1, x1*x2

DesignMatrix with shape (5, 2)
        x1      x1:x2
  10.54881   68.13417
  10.71519   46.88830
  10.60276   94.55258
  10.54488  101.61711
  10.42365   39.96862
  Terms:
    'x1' (column 0)
    'x1:x2' (column 1)

---

### 03. Dmatrix 변환

dmatrix에서는 일반적인 수학 변환(transform)도 가능

#### 1. 일반적인 수학변환

In [13]:
dmatrix("x1 + np.log(np.abs(x2))")

DesignMatrix with shape (5, 3)
  Intercept        x1  np.log(np.abs(x2))
          1  10.54881             1.86547
          1  10.71519             1.47611
          1  10.60276             2.18804
          1  10.54488             2.26557
          1  10.42365             1.34402
  Terms:
    'Intercept' (column 0)
    'x1' (column 1)
    'np.log(np.abs(x2))' (column 2)

In [14]:
def doubleit(x):
    return 2 * x

dmatrix("doubleit(x1)")

DesignMatrix with shape (5, 2)
  Intercept  doubleit(x1)
          1      21.09763
          1      21.43038
          1      21.20553
          1      21.08977
          1      20.84731
  Terms:
    'Intercept' (column 0)
    'doubleit(x1)' (column 1)

#### 2. dmatrix내 함수 종류

* center(x): 평균 제거
* standardize(x): 평균 제거 및 표준편차로 스케일링
* scale(x): standardize(x) 과 같음

In [15]:
dmatrix("center(x1) + standardize(x1) + scale(x2)")

DesignMatrix with shape (5, 4)
  Intercept  center(x1)  standardize(x1)  scale(x2)
          1    -0.01825         -0.19319   -0.07965
          1     0.14813          1.56828   -0.97279
          1     0.03570          0.37799    0.97458
          1    -0.02218         -0.23480    1.28282
          1    -0.14341         -1.51828   -1.20495
  Terms:
    'Intercept' (column 0)
    'center(x1)' (column 1)
    'standardize(x1)' (column 2)
    'scale(x2)' (column 3)

---

### 04. 변수 보호

- 변수 변환 이외에도 `모형 정의 문자열에 연산기호를 넣어` 연산한 값을 만드는 것도 가능


- 모형정의 연산자와 혼동되지 않도록 I( ) 연산자를 추가해야 함

In [16]:
# 연산 전 - 단순히 옆에 붙여진 것을 확인
dmatrix("x1 + x2") 

DesignMatrix with shape (5, 3)
  Intercept        x1       x2
          1  10.54881  6.45894
          1  10.71519  4.37587
          1  10.60276  8.91773
          1  10.54488  9.63663
          1  10.42365  3.83442
  Terms:
    'Intercept' (column 0)
    'x1' (column 1)
    'x2' (column 2)

In [17]:
# 연산 후 - I가 +의 역할을 보호하여 연산을 수행하게 함
dmatrix("I(x1 + x2)")

DesignMatrix with shape (5, 2)
  Intercept  I(x1 + x2)
          1    17.00775
          1    15.09106
          1    19.52049
          1    20.18151
          1    14.25807
  Terms:
    'Intercept' (column 0)
    'I(x1 + x2)' (column 1)

In [18]:
# 뺄셈도 가능
dmatrix("I(x1 - x2)")

DesignMatrix with shape (5, 2)
  Intercept  I(x1 - x2)
          1     4.08987
          1     6.33932
          1     1.68503
          1     0.90826
          1     6.58924
  Terms:
    'Intercept' (column 0)
    'I(x1 - x2)' (column 1)

---

### 05. 다항 선형 회귀 (polynomial regression)

I( ) 연산자를 통해 가능

In [19]:
dmatrix("x1 + I(x1**2) + I(x1**3) + I(x1**4)") # x1, x1제곱, x1세제곱, x1네제곱을 구함

DesignMatrix with shape (5, 5)
  Intercept        x1  I(x1 ** 2)  I(x1 ** 3)   I(x1 ** 4)
          1  10.54881   111.27747  1173.84524  12382.67452
          1  10.71519   114.81528  1230.26750  13182.54925
          1  10.60276   112.41859  1191.94772  12637.93965
          1  10.54488   111.19456  1172.53366  12364.23047
          1  10.42365   108.65258  1132.55698  11805.38301
  Terms:
    'Intercept' (column 0)
    'x1' (column 1)
    'I(x1 ** 2)' (column 2)
    'I(x1 ** 3)' (column 3)
    'I(x1 ** 4)' (column 4)

---

### 06. 카테고리 변수 인코딩

데이터로 문자열이 오는 경우에는 카테고리 값으로 인정하여 One-Hot-Encoding 방식의 인코딩을 실시

#### 1. 문자로 표시된 카테고리

In [20]:
# 카테고리 데이터 생성
df3 = pd.DataFrame(["A", "B", "C", "D"], columns=["x3"])
df3

,x3
0,A
1,B
2,C
3,D


In [21]:
# One-Hot-Encoding 실시
dmatrix("x3 - 1", df3)

DesignMatrix with shape (4, 4)
  x3[A]  x3[B]  x3[C]  x3[D]
      1      0      0      0
      0      1      0      0
      0      0      1      0
      0      0      0      1
  Terms:
    'x3' (columns 0:4)

$\rightarrow$ 주의해야 할 점은 augmentation을 없애줘야 오류 발생없이 인코딩이 실행된다는 점이다

In [22]:
dmatrix("x3", df3) # 0,0,0이 발생 (정의에 맞지 않음)

DesignMatrix with shape (4, 4)
  Intercept  x3[T.B]  x3[T.C]  x3[T.D]
          1        0        0        0
          1        1        0        0
          1        0        1        0
          1        0        0        1
  Terms:
    'Intercept' (column 0)
    'x3' (columns 1:4)

#### 2. 숫자로 표시된 카테고리 - C( ) 연산자를 이용

In [23]:
df4 = pd.DataFrame([1, 3, 4, 2], columns=["x4"])
df4

,x4
0,1
1,3
2,4
3,2


In [24]:
# column name에 C( )을 이용해 카테고리 변수라 지정
dmatrix("C(x4) + 0", df4) 

DesignMatrix with shape (4, 4)
  C(x4)[1]  C(x4)[2]  C(x4)[3]  C(x4)[4]
         1         0         0         0
         0         0         1         0
         0         0         0         1
         0         1         0         0
  Terms:
    'C(x4)' (columns 0:4)

---

### 07. OLS 모형 정의 문자열

- 선형회귀분석을 위한 OLS 클래스에는 모형 정의 문자열을 사용할 수 있는 from_formula라는 메서드가 존재


- 사용자가 데이터 행렬을 직접 정의하지 않고 모형 정의 문자열만으로 선형회귀모형을 만드는 것이 가능

```
model2 = sm.OLS.from_formula("y ~ x1 + x2", data=df)
```

- y : 종속변수
- x1 : 독립변수 1
- x2 : 독립변수 2

In [25]:
df # 데이터 생성

,x1,x2,y
0,10.548814,6.458941,23.610739
1,10.715189,4.375872,20.921207
2,10.602763,8.917730,29.199261
3,10.544883,9.636628,29.939813
4,10.423655,3.834415,18.536348


#### 1. 직접 데이터 행렬을 만드는 경우

In [26]:
dfy = df.iloc[:, -1]
dfX = sm.add_constant(df.iloc[:, :-1]) # augmentation 생성
model1 = sm.OLS(dfy, dfX)

In [27]:
print(model1.fit().summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.995
Method:                 Least Squares   F-statistic:                     406.1
Date:                Sun, 04 Mar 2018   Prob (F-statistic):            0.00246
Time:                        13:11:46   Log-Likelihood:                0.41801
No. Observations:                   5   AIC:                             5.164
Df Residuals:                       2   BIC:                             3.992
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -41.9480     17.631     -2.379      0.1

/usr/local/lib/python3.6/site-packages/statsmodels/stats/stattools.py:72: ValueWarning: omni_normtest is not valid with less than 8 observations; 5 samples were given.
  "samples were given." % int(n), ValueWarning)


#### 2. 모형 정의 문자열을 사용하는 경우 (OLS.from_formula)

In [28]:
model2 = sm.OLS.from_formula("y ~ x1 + x2", data=df)

In [29]:
print(model1.fit().summary()) # 한줄로 정리됨

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.995
Method:                 Least Squares   F-statistic:                     406.1
Date:                Sun, 04 Mar 2018   Prob (F-statistic):            0.00246
Time:                        13:11:46   Log-Likelihood:                0.41801
No. Observations:                   5   AIC:                             5.164
Df Residuals:                       2   BIC:                             3.992
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -41.9480     17.631     -2.379      0.1

/usr/local/lib/python3.6/site-packages/statsmodels/stats/stattools.py:72: ValueWarning: omni_normtest is not valid with less than 8 observations; 5 samples were given.
  "samples were given." % int(n), ValueWarning)


$\rightarrow$ 동일한 Performance를 내지만 OLS.from_formula가 훨씬 편함